In [ ]:
import torch
import torch.nn as nn
from .utils import load_state_dict_from_url
from typing import Any


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        #특징추출
        self.features = nn.Sequential(
            #Conv1
            #input channel : 3 (rgb 3채널) , output channel : 64, kernel_size : 11, stride : 4, padding : 2
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True), # inplace=True 하면, inplace 연산을 수행함(교체), inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미
            #Max Pool1
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv2
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            #Max Pool2
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ##Conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Max Pool3
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        #분류
        #시퀀셜 형태로 layer을 쌓을거임
        self.classifier = nn.Sequential(
            #드롭아웃 
            nn.Dropout(), 
            nn.Linear(256 * 6 * 6, 4096), #fully connected layer를 linear로 쌓음
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes), #최종 분류 층, 사용하려는 전체 클래스 개수와 맞춰줌(output num_class)
        )
        
        #이미지가 들어왔을때 어떤 순서를 거쳐 진행되는지 forward함수로 지정해줌
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #특징 추출 부분
        x = self.features(x)
       
        x = self.avgpool(x)
        #output shape : (batch size * 256(channel), 6, 6)
        #Flatten
        x = torch.flatten(x, 1) #한 배치마다 특징을 펼쳐서
        #output shape (batch_size, 256 * 6* 6)
        #분류 
        x = self.classifier(x) #분류
        return x

#init으로 사전학습된 모델을 갖고올지 말지 boolean, 
def alexnet(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> AlexNet:
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    #모델에 alexnet넣어줌
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model